In [ ]:
# Instala as dependências
!pip install -r ../../../requirements.txt

In [ ]:
import pandas as pd
from sklearn.svm import OneClassSVM
import matplotlib.pyplot as plt

In [ ]:
dados = pd.read_csv('../../../data/tratado/dados_cvm_tratados.csv')

In [ ]:
# Evitando logs no plot de gráficos
pd.options.mode.chained_assignment = None

# *OneClassSVM*

No desafio de identificar e prever problemas relacionados à provisão de perdas em Fundos de Investimento em Direitos Creditórios, é essencial explorar métodos que permitam detectar anomalias. Entre as abordagens consideradas, destaca-se o uso do modelo *One-Class SVM*, devido à sua eficácia em identificar anomalias em conjuntos de dados complexos, como o cenário apresentado.

O modelo *One-Class SVM* é uma técnica de detecção de anomalias que se baseia em uma abordagem de aprendizado de máquina para distinguir observações normais de observações anômalas em um conjunto de dados. Ele é particularmente adequado para problemas em que a maioria dos dados é composta por observações normais e as anomalias são raras. Aqui, ele pode ser aplicado para prever a ocorrência de problemas relacionados à insuficiência ou ausência de provisão para perdas em FIDCs.

O funcionamento do modelo *One-Class SVM* envolve a criação de uma "fronteira" em torno da maioria dos dados normais, tentando maximizar a margem entre essa fronteira e as observações normais. Tudo o que estiver fora dessa fronteira é considerado uma anomalia. Em outras palavras, o modelo tenta encontrar um hiperplano que melhor separe os dados normais das anomalias, minimizando o número de observações anômalas que caem do lado errado do hiperplano.

Usaremos os resultados obtidos com o modelo *One-Class SVM* para destacar e apresentar as observações consideradas como outliers nos gráficos dos Fundos de Investimento em Direitos Creditórios (FIDCs) que demonstrarem a maior quantidade de SK_Documentos classificados como anomalias. Em seguida, compararemos esses resultados com os obtidos pelo nosso modelo principal, o Isolation Forest. Essa abordagem permitirá uma análise comparativa abrangente, ajudando-nos a avaliar a eficácia de ambos os modelos na detecção de problemas relacionados à provisão de perdas em FIDCs e, potencialmente, identificar áreas em que um modelo supera o outro em termos de sensibilidade e especificidade na detecção de anomalias.

In [ ]:
dados = dados.drop(columns=['Data_Competencia'])


In [ ]:
# Lista de carteiras com índices correspondentes à classificação
classificacao_encoding = [
    'SetorPublico',
    'Agronegocio',
    'Cartao',
    'Comercial',
    'Imobiliario',
    'Financeiro',
    'Industrial',
    'Factoring',
    'Multimercado',
    'Servicos',
    'AcoesJudiciais'
]

In [ ]:
# Separando os dados por carteira
dados_classificacao = {
    'Setor Público': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('SetorPublico')],
    'Agronegócio': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Agronegocio')],
    'Cartão': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Cartao')],
    'Comercial': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Comercial')],
    'Imobiliário': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Imobiliario')],
    'Financeiro': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Financeiro')],
    'Industrial': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Industrial')],
    'Factoring': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Factoring')],
    'Multimercado': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Multimercado')],
    'Serviços': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Servicos')],
    'Ações Judiciais': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('AcoesJudiciais')]
}

# Construção do Modelo

In [ ]:
def detectar_outliers(dados, nu=0.05, kernel='rbf'):
    clf = OneClassSVM(gamma='auto', nu=nu, kernel=kernel)
    clf.fit(dados)

    # Predição dos outliers
    pred = clf.predict(dados)
    dados.loc[:, 'anomalias'] = pred

    # Filtrando os outliers
    outliers = dados.loc[dados['anomalias'] == -1]

    # Contagem dos outliers para cada valor único em 'ID_Participante'
    top_outliers = outliers['ID_Participante'].value_counts().head(10)

    return top_outliers

# Analisando os Resultados

In [ ]:
dados_raw = pd.read_csv('../../../data/tratado/dados_cvm_tratados.csv')

def pipeline_analise(classificacao, numero_outliers=2, kernel='rbf'):
    top_outliers = detectar_outliers(dados_classificacao[classificacao], kernel=kernel)
    for i in range(min(numero_outliers, len(top_outliers))):
        df_analise = dados_raw.query(f'ID_Participante == {top_outliers.index[i]}')

        fig, ax = plt.subplots(figsize=(15, 5))
        ax.plot(df_analise['Data_Competencia'], df_analise['Inadimplencia_Total'], label='Inadimplencia_Total')
        ax.plot(df_analise['Data_Competencia'], df_analise['Provisao_Total'], label='Provisao_Total')
        ax.set_title(f'Inadimplencia_Total e Provisao_Total - ID_Participante: {top_outliers.index[i]} - Carteira: {classificacao}')
        ax.legend()
        plt.show()

# Otimização de hiperparâmetros do *OneClassSVM*
A presente seção se concentra na otimização dos hiperparâmetros associados ao algoritmo *One-Class Support Vector Machine (OneClassSVM)*. Este processo é realizado de forma iterativa, preservando o valor do hiperparâmetro "nu" estabelecido em 0.05, o qual representa o nível de contaminação. Importa salientar que este valor já foi submetido a testes prévios e validado no contexto do algoritmo *Isolation Forest*.
O procedimento de otimização é conduzido através de uma iteração sistemática sobre diferentes *kernels* disponíveis. A escolha do *kernel* apropriado é de suma importância, uma vez que influencia diretamente o desempenho do modelo OneClassSVM. Os *kernel*s submetidos a análise abrangem:

*Kernel RBF (Radial Basis Function)*: Este *kernel* utiliza uma função de base radial, sendo amplamente empregado devido à sua flexibilidade e capacidade de modelar relações complexas nos dados. Ele é especialmente útil quando não se tem conhecimento prévio sobre a distribuição dos dados.

*Kernel Linear*: O *kernel* linear é uma escolha fundamental quando se presume que os dados possuam uma estrutura linear subjacente. Ele é eficaz na detecção de anomalias em conjuntos de dados que apresentam uma separação linear entre as classes normais e as anomalias.

*Kernel Polinomial (Poly)*: O *kernel* polinomial é empregado quando se suspeita que os dados possam seguir uma relação polinomial. Ele é adequado para modelar padrões mais complexos do que o *kernel* linear, mas menos complexos do que o *kernel* RBF.

*Kernel* Sigmóide: O *kernel* sigmóide é útil em situações onde se acredita que os dados possam seguir uma relação sigmóide. Ele pode ser empregado quando a separação entre classes não é necessariamente linear, mas ainda assim não segue a complexidade do *kernel* RBF.

A avaliação dos resultados obtidos com cada *kernel* se baseia na análise das métricas relacionadas à inadimplência e provisão, visualizadas em gráficos. O objetivo principal é identificar o modelo que apresente os resultados mais anômalos. Nesse contexto, a anomalia se refere a observações que divergem significativamente do comportamento esperado, em que as provisões acompanham ou superam a inadimplência, sendo de interesse primordial na análise de detecção de anomalias.

In [ ]:
available_kernels = [
    'rbf',
    'linear',
    'poly',
    'sigmoid'
]

for kernel in available_kernels:
    for item in dados_classificacao.keys():
      print(item)
      pipeline_analise(item, numero_outliers=3, kernel=kernel)

# Resultados da Análise
Após uma análise criteriosa dos resultados obtidos na otimização de hiperparâmetros do *OneClassSVM*, foi observado que os *kernel*s mais promissores foram o linear e o RBF (*Radial Basis Function*). Ambos os *kernels* demonstraram capacidade de modelar efetivamente a relação entre as observações, destacando-se na detecção de anomalias.
No entanto, após uma análise aprofundada das características dos dados e das particularidades do problema em questão, a decisão foi tomada de manter o *kernel* RBF como a escolha preferencial para a modelagem do *OneClassSVM*. A razão subjacente a essa escolha reside na crença de que os dados em questão apresentam uma complexidade maior do que pode ser adequadamente capturada pelo *kernel* linear.
Vale ressaltar que tanto o kernel linear quanto o polinomial produziram resultados bastante semelhantes, sugerindo que o modelo reduziu a relevância das variáveis com grau superior a um, indicando uma limitação na detecção de relações polinomiais claras.
O *kernel* RBF é conhecido por sua capacidade de modelar relações não lineares complexas nos dados. Ele possui uma flexibilidade inerente que lhe permite se adaptar a padrões de alta complexidade, o que o torna uma escolha sólida quando se suspeita que as anomalias possam ter padrões não lineares menos óbvios.
Além disso, a manutenção do *kernel* RBF também pode ser justificada pelo fato de que a detecção de anomalias muitas vezes exige a exploração de relações de alta dimensionalidade e não linearidade nos dados. A capacidade do *kernel* RBF de lidar com esses cenários desafiadores é uma vantagem considerável.


Apesar da decisão de manter o *kernel* RBF no *OneClassSVM*, vale ressaltar que o modelo de detecção de anomalias que se destacou durante o processo de ideação e validação de modelos foi o *Isolation Forest*. Este modelo se mostrou especialmente eficaz na identificação de anomalias relacionadas aos FIDCS.
Portanto, embora o *OneClassSVM* tenha apresentado resultados interessantes e tenha sido objeto de otimização cuidadosa, o *Isolation Forest* se destacou como a escolha preferencial devido à sua habilidade de identificar anomalias de forma mais precisa e eficiente, especialmente em cenários que envolvem grandes volumes de dados ou relações de alta dimensionalidade. Essa preferência reflete a busca por resultados mais alinhados com as expectativas e requisitos específicos do problema relacionado a provisões e inadimplências.